In [1]:
import pandas as pd
import numpy as np
import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [2]:
from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/new_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels20.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno

In [4]:
import os
# rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100"

objective="ndcg_cut_5"
for i, (subdir, dirs, files) in enumerate(os.walk(rootdir)):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm = pt.BatchRetrieve(index, wmodel="DirichletLM", controls={"dirichletlm.mu" : 2500})
            tuned_dlm, parameters = pt.KFoldGridSearch(
                dlm,
                {dlm: {"dirichletlm.mu" : list(range(0,10000,250))}},
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels, 
                                names=['tuned_dlm_unjudged'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics20, qrels, 
                                names=['tuned_dlm_unjudged20'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics21, qrels, 
                                names=['tuned_dlm_unjudged21'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
                                    

            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100
Fold 1
Best ndcg_cut_5 is 0.652732
Best setting is ['BR(DirichletLM) dirichletlm.mu=3000']
Fold 2
Best ndcg_cut_5 is 0.467390
Best setting is ['BR(DirichletLM) dirichletlm.mu=4250']
                 name  ndcg_cut_5  ndcg_cut_25     bpref
0  tuned_dlm_unjudged    0.675397     0.576817  0.508813
                   name  ndcg_cut_5  ndcg_cut_25     bpref
0  tuned_dlm_unjudged20    0.670699     0.524215  0.421595
                   name  ndcg_cut_5  ndcg_cut_25     bpref
0  tuned_dlm_unjudged21    0.680001     0.628368  0.594287


In [5]:
'''
Example of simple tuning dlm and its not saving to the folder 
'''

import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
metrics="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm = pt.BatchRetrieve(index, wmodel="DirichletLM", controls={"dirichletlm.mu" : 2500})
            tuned_dlm, parameters = pt.KFoldGridSearch(
                dlm,
                {dlm: {"dirichletlm.mu" : list(range(0,10000,250))}},
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_dlm_unjudged'],
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics20, qrels, names=[folders[i] + '_tuned_dlm_unjudged20'],
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            print(pt.Experiment([tuned_dlm[tuned_dlm['docno'].isin(docnos)]], topics21, qrels, names=[folders[i] + '_tuned_dlm_unjudged21'],
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            

            i += 1
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.607973
Best setting is ['BR(DirichletLM) dirichletlm.mu=3750']
Fold 2
Best ndcg_cut_5 is 0.391740
Best setting is ['BR(DirichletLM) dirichletlm.mu=5750']
                             name  ndcg_cut_5  ndcg_cut_25     bpref
0  conclusions_tuned_dlm_unjudged    0.688756     0.637897  0.592431
                               name  ndcg_cut_5  ndcg_cut_25     bpref
0  conclusions_tuned_dlm_unjudged20    0.662305     0.566789  0.462891
                               name  ndcg_cut_5  ndcg_cut_25    bpref
0  conclusions_tuned_dlm_unjudged21    0.714678     0.707583  0.71938

/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pool_topicspd_index
Fold 1
Best ndcg_cut_5 is 0.624379
Best setting is ['BR(DirichletLM) dirichletlm.mu=2250']
Fold 2
Best ndcg_cut_5 is 0.372219
Best setting is ['BR(DirichletLM) dirichletlm.